## 1024

In [5]:
# -*- coding:utf-8 -*-
#  author: yukun
import requests
import os
import time
import re
from bs4 import BeautifulSoup
from lxml import etree
from tqdm import tqdm

# 发出请求获得HTML源码
def get_html(url):
    # 指定一个浏览器头
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    # 代理，免费的代理只能维持一会可能就没用了，自行更换
    proxies = {'http': '111.23.10.27:8080'}
    try:
        # Requests库的get请求
        resp = requests.get(url, headers=headers)
    except:
        # 如果请求被阻，就使用代理
        resp = requests.get(url, headers=headers, proxies=proxies)
        print('代理登陆中')
    return resp
 
# 创建文件夹的函数，保存到E盘
def mkdir(root_path,path):    
    
    isExists = os.path.exists(os.path.join(root_path,path)) #os.path.exists(name)判断是否存在路径
    # 如果不存在
    if not isExists:
        #print('makedir', path)
        # 创建文件夹
        os.makedirs(os.path.join(root_path, path))# os.path.join(path, name)连接目录与文件名
        # 切换到创建的文件夹
        os.chdir(os.path.join(root_path, path))
        return True
    # 如果存在了就返回False
    else:
        print(path, 'already exists')
        return False
    
# 获得所有页面
def all_page(base_url,son_url):
    
    resp = get_html(base_url+son_url)#获取网页内容
    resp.encoding='utf-8'#选择编码方式，不需要返回值
    web_data = resp.text #对解码完成的网页选择文本形式，保持到web_data中
    html = etree.HTML(web_data)# 将页面转换成文档树
    link = html.xpath('//h3/a/@href')#用xpath选取所有子节点中名称为h3的节点，
    #并选取对于直接子节点a的属性进行保存，这里的是子网页的链接
    link = list(filter(lambda x: x.find('html') != -1 ,link))
    #用过滤器看名称是不是html_data/14/1910/4390161.html类型的 
    #fileter需要用list包装返回的才是列表，否则直接返回一个地址
    urllist = []
    for x in link:
        urllist.append(base_url+ str(x))
        
#     soup = BeautifulSoup(get_html(base_url).text, 'lxml')
#     urllist = []
#     t1 = soup.find_all('a')
#     #print(t1)
#     for t2 in t1:
#         t3 = t2.get('href') 
#         if "html_data" in str(t3):
#             urllist.append('http://p3.csgfnmdb.com/pw/'+ str(t3))

    urllist=list(set(urllist))#set去重,去重后再用list包装
    #print(urllist)
    
    return urllist
# 保存图片函数，传入的参数是一页所有图片url集合



def download(imgs_link):
    for url in tqdm(imgs_link):
        # 判断url是否完整
        if url[0:5] == 'http:':
            img_url = url
        else:
            img_url = 'http:' + url
        filename = img_url.split('/')[-1]
        tpye = filename.split('.')[-1]
        if  tpye =='jpg':
            # 保存图片
            with open(filename, 'wb') as f:
                # 直接过滤掉保存失败的图片，不终止程序
                try:
                    f.write(get_html(img_url).content)
                    #print('Sucessful image:',filename)
                except:
                    print('Failed:',filename)
        
# # 获得图片地址调用download函数进行下载
# def get_imgs():
#     # 调用函数获得所有页面
#     #urllist = all_page()
#     for url in all_page():
#         new = get_html(url)
#         new.encoding='utf-8'
#         soup = BeautifulSoup(new.text, 'lxml')
#         path = soup.find('title').text.split('|')[0]
#         print(path)
    
#         # 创建文件夹的函数
#         mkdir(path)
#         # 调用请求函数获得HTML源码
#         html = get_html(url).text
#         # 使用lxml解析器，也可以使用html.parser
#         soup = BeautifulSoup(html, 'lxml')
#         #print(soup)
#         # css选择器
#         allimgs = soup.select('img')
#         #print(allimgs)        
#         # 调用download函数下载保存
#         download(allimgs)
#     # 执行完毕打出ok
#     print('ok')
    
    
if __name__ == '__main__':
    # 计时
    t1 = time.time()
    base_path = 'D:\\1024\\zip' 
    base_url = 'http://x1.wcnnedrb.top/pw/'
    son_url = 'thread.php?fid=15'
    urllist = all_page(base_url,son_url)#获取子页面所有链接

    for url in urllist[:]:
        time.sleep(1)#每爬完一个延迟1秒
        #print(url)
        resp = get_html(url)#返回结果
        resp.encoding='utf-8' 
        web_data = resp.text
        html = etree.HTML(web_data)
        imgs_link = html.xpath('//div[@class = "tpc_content"]//img/@src')#获取图片链接
        title = html.xpath('//h1[@class="b"]//span[@id="subject_tpc"]/text()')#获取页面名称作为保存路径
        print(title)
        mkdir(base_path,title[0])#创建文件夹
        download(imgs_link)#从图片链接进行下载

        
    print(time.time() - t1)

['朋友家底厚，爱摄影，身边自然少不了美女，这个是他调教的最听话的一个！[15P]']
朋友家底厚，爱摄影，身边自然少不了美女，这个是他调教的最听话的一个！[15P] already exists


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:31<00:00,  1.60s/it]


['嫩穴就像个小女孩一样，粉嫩可口[12P]']


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.62s/it]


['勾引同事[11P]']


...

['听主人话的女仆[12P]']


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:21<00:00,  2.42s/it]


['身材丰满，逼也肥[12P]']


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:19<00:00,  1.19s/it]


1280.3548254966736
